In [88]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)


In [170]:
df = pd.read_csv('../data/clean_train_all_cols_chase.csv', parse_dates=['timestamp'], index_col="id")  
# test = pd.read_csv('../data/clean_test_all_cols_chase.csv', parse_dates=['timestamp'], index_col="id")

In [171]:
macro = pd.read_csv('../data/macro_chase.csv')
macro['quarter'] = pd.PeriodIndex(macro['Unnamed: 0'], freq='Q').strftime('Q%q-%y')
df['quarter'] = pd.PeriodIndex(df['timestamp'], freq='Q').strftime('Q%q-%y')

df = pd.merge(df,macro[['quarter','nominal_index']], how="left", on="quarter").reset_index(drop=True).set_index(df.index)

cols = ['material', 'build_year', 'num_room', 'state']
print df.shape
df.drop(cols,axis=1,inplace=True)
print df.shape

(29096, 299)
(29096, 295)


In [172]:
# drop all columns that don't have atleast 28000 values
df.dropna(thresh=28000, axis=1, inplace=True)
print df.shape

(29096, 260)


In [175]:
df.dropna(axis=0,inplace=True)
print df.shape

(28090, 260)


In [176]:
df.sub_area = df.sub_area.apply(lambda x: x.replace(" ", "_"))
df.sub_area = df.sub_area.apply(lambda x: x.replace("'", ""))
df.sub_area = df.sub_area.apply(lambda x: x.replace("-", ""))

In [177]:
df['kitch_to_life'] = df.kitch_sq / df.life_sq
df['life_to_full'] = df.life_sq / df.full_sq
df['walk_up_penalty'] = 0
df.loc[(df.floor>4) & (df.max_floor < 6),'walk_up_penalty'] = 1 

In [178]:
df['price_doc'] = df.price_doc / df.nominal_index
df['price_full'] = df.price_doc / df.full_sq
# df['log_price'] = np.log(df.price_doc)

# creating a variable for how expensive the property is
df['price_class'] = pd.qcut(df.price_full, 5, labels=["E","D","C","B","A"])

## Decision Tree

In [204]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [182]:
drop_cols = ['price_doc','price_full','price_class','okurg_district','sub_area','quarter','quarter']
keep_cols = ['price_class','kremlin_km','floor','full_sq','product_type','full_sq','kitch_sq','max_floor','life_sq']


# df_obj = df.drop(drop_cols,axis=1).select_dtypes(include=['object'])
# df_num = df.drop(drop_cols,axis=1).select_dtypes(exclude=['object'])

df = df[keep_cols]
df_obj = df.select_dtypes(include=['object'])
df_num = df.select_dtypes(exclude=['object'])

dummies = pd.get_dummies(df_obj)
df_all = pd.concat([df_num,dummies],axis=1)

In [202]:
tree_clf = DecisionTreeClassifier(max_depth=10)

x = df_all.drop('price_class',axis=1)
y = df_all.price_class
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
tree_clf.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [203]:
pd.DataFrame({'importance': tree_clf.feature_importances_},index=x.columns.tolist())


,importance
kremlin_km,0.629106
floor,0.027680
full_sq,0.068296
full_sq,0.049865
kitch_sq,0.043461
max_floor,0.068010
life_sq,0.050816
product_type_Investment,0.039934
product_type_OwnerOccupier,0.022831


In [201]:
tree_clf.decision_path

<bound method DecisionTreeClassifier.decision_path of DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')>

In [187]:
from sklearn.tree import export_graphviz

from IPython.display import Image  
dot_data = tree.export_graphviz(tree_clf, out_file=None, 
                         feature_names=tree_clf.feature_names,  
                         class_names=tre.target_names,  
                         filled=True, rounded=True,  
                         special_characters=True) 
graph = pydotplus.graph_from_dot_data(dot_data)  
Image(graph.create_png())  

SyntaxError: invalid syntax (<ipython-input-187-f612467ec71c>, line 7)